# Cognitive Decision Graph (CDG) Demo

This notebook demonstrates the CDG data structure for analyzing Python programs at a cognitive/semantic level.


In [1]:
from cdg import (
    CDGBuilder, build_cdg, compare_programs, print_cdg,
    programs_equivalent, messages_semantically_similar,
    Interval, Region,
    InputNode, ControlNode, BranchNode, ActionNode, BodyNode
)


## Example 1: Basic Astronaut Height Checker

This is the first example program with if/elif/else structure.


In [2]:
code_1 = '''
def main():
    height = float(input("Enter your height in meters: "))
    if height < 1.6:
        print("Below minimum astronaut height")
    elif 1.6 < height < 1.9:
        print("Correct height to be an astronaut")
    else:
        print("Above maximum astronaut height")
'''

print("=== Program 1 CDG ===")
print_cdg(code_1)


=== Program 1 CDG ===


BodyNode(main)
  InputNode(type=float, prompt="Enter your height in meters: ", category=height)
  ControlNode(type=if, branches=3)
    BranchNode(if/elif, region=(-∞, 1.6))
      ActionNode(print, msg="Below minimum astronaut height")
    BranchNode(if/elif, region=(1.6, 1.9))
      ActionNode(print, msg="Correct height to be an astronaut")
    BranchNode(else, region={1.6} ∪ [1.9, ∞))
      ActionNode(print, msg="Above maximum astronaut height")


## Example 2: While Loop Variant

Same logic but wrapped in a `while True` loop with different condition ordering.


In [6]:
code_2 = '''
def main():
    height = float(input("Enter your height in meters: "))
    if height > 1.6 and height < 1.9:
        print("Correct height to be an astronaut")
    elif height <= 1.6:
        print("Below minimum astronaut height")
    elif height >= 1.9:
        print("Above maximum astronaut height")
'''

print("=== Program 2 CDG ===")
print_cdg(code_2)


=== Program 2 CDG ===
BodyNode(main)
  InputNode(type=float, prompt="Enter your height in meters: ")
  ControlNode(type=if, branches=3)
    BranchNode(if/elif, region=(1.6, 1.9))
      ActionNode(print, msg="Correct height to be an astronaut")
    BranchNode(if/elif, region=(-∞, 1.6])
      ActionNode(print, msg="Below minimum astronaut height")
    BranchNode(if/elif, region=[1.9, ∞))
      ActionNode(print, msg="Above maximum astronaut height")


## Compare the Two Programs


In [5]:
similarity = compare_programs(code_1, code_2)

print("\n=== Similarity Analysis ===")
for metric, score in similarity.items():
    print(f"  {metric:15}: {score:.2f}")



=== Similarity Analysis ===
  structural     : 0.45
  behavioral     : 1.00
  regional       : 0.20
  input          : 1.00
  action         : 1.00
  overall        : 0.49


## Interval Arithmetic Demo

The Region class handles complex interval operations.


In [ ]:
# Create intervals
below_1_6 = Interval(float('-inf'), 1.6, False, False)  # (-inf, 1.6)
above_1_9 = Interval(1.9, float('inf'), True, False)     # [1.9, inf)

print(f"Below 1.6: {below_1_6}")
print(f"Above 1.9: {above_1_9}")

# Create regions from constraints
region_lt_1_6 = Region.from_constraint('<', 1.6)
region_gt_1_6 = Region.from_constraint('>', 1.6)
region_lt_1_9 = Region.from_constraint('<', 1.9)

print(f"\nRegion (height < 1.6): {region_lt_1_6}")
print(f"Region (height > 1.6): {region_gt_1_6}")

# Intersection (AND)
correct_height = region_gt_1_6.intersect(region_lt_1_9)
print(f"\nCorrect height (1.6 < h < 1.9): {correct_height}")

# Complement (ELSE)
not_correct = correct_height.complement()
print(f"Not correct height: {not_correct}")


## Program Equivalence Check (Boolean)

The `programs_equivalent` function returns True/False indicating whether two programs are semantically equivalent. Unlike `compare_programs` which gives similarity scores, this is a strict equivalence check.

Key features:
- Returns False immediately upon finding any difference
- Uses semantic message comparison (not exact word matching)
- Order-independent branch comparison (if/elif in different order is OK if regions match)


In [3]:
# Example 1: Equivalent programs (different variable names, same semantics)
code_equiv_1 = '''
def main():
    h = float(input("Height: "))
    if h > 1.6 and h < 1.9:
        print("OK")
    else:
        print("Not OK")
'''

code_equiv_2 = '''
def main():
    height = float(input("Height: "))
    if 1.6 < height < 1.9:
        print("OK")
    else:
        print("Not OK")
'''

is_equiv, reason = programs_equivalent(code_equiv_1, code_equiv_2)
print("=== Example 1: Different variable names, same logic ===")
print(f"Equivalent: {is_equiv}")
if not is_equiv:
    print(f"Reason: {reason}")


=== Example 1: Different variable names, same logic ===
Equivalent: True


In [4]:
# Example 2: Equivalent with semantic message matching
# "Above max astronaut height" ~ "Above maximum height" (both say "above")
code_sem_1 = '''
def main():
    height = float(input("Enter your height: "))
    if height >= 1.9:
        print("Above max astronaut height")
    elif height <= 1.6:
        print("Below minimum height")
    else:
        print("Correct height to be an astronaut")
'''

code_sem_2 = '''
def main():
    h = float(input("Enter your height: "))
    if h >= 1.9:
        print("Above maximum height")
    elif h <= 1.6:
        print("Below min height")
    else:
        print("Correct height")
'''

is_equiv, reason = programs_equivalent(code_sem_1, code_sem_2)
print("\n=== Example 2: Semantic message similarity ===")
print(f"Equivalent: {is_equiv}")
if not is_equiv:
    print(f"Reason: {reason}")

# Show what the semantic similarity function does
print("\nSemantic message comparisons:")
print(f"  'Above max astronaut height' ~ 'Above maximum height': {messages_semantically_similar('Above max astronaut height', 'Above maximum height')}")
print(f"  'Below minimum height' ~ 'Below min height': {messages_semantically_similar('Below minimum height', 'Below min height')}")
print(f"  'Correct height to be an astronaut' ~ 'Correct height': {messages_semantically_similar('Correct height to be an astronaut', 'Correct height')}")



=== Example 2: Semantic message similarity ===
Equivalent: True

Semantic message comparisons:
  'Above max astronaut height' ~ 'Above maximum height': True
  'Below minimum height' ~ 'Below min height': True
  'Correct height to be an astronaut' ~ 'Correct height': True


In [5]:
# Example 3: NOT equivalent - different messages (above vs below)
code_diff_1 = '''
def main():
    height = float(input("Height: "))
    if height >= 1.9:
        print("Above maximum height")
    else:
        print("OK")
'''

code_diff_2 = '''
def main():
    height = float(input("Height: "))
    if height >= 1.9:
        print("Below minimum height")  # Wrong direction!
    else:
        print("OK")
'''

is_equiv, reason = programs_equivalent(code_diff_1, code_diff_2)
print("\n=== Example 3: Different message semantics (above vs below) ===")
print(f"Equivalent: {is_equiv}")
if not is_equiv:
    print(f"Reason: {reason}")

# Show why they're different
print(f"\nSemantic check: 'Above maximum height' ~ 'Below minimum height': {messages_semantically_similar('Above maximum height', 'Below minimum height')}")



=== Example 3: Different message semantics (above vs below) ===
Equivalent: False
Reason: Child 1: No matching branch for region [1.9, ∞)

Semantic check: 'Above maximum height' ~ 'Below minimum height': False


In [6]:
# Example 4: NOT equivalent - different thresholds
code_thresh_1 = '''
def main():
    height = float(input("Height: "))
    if height < 1.6:
        print("Too short")
    else:
        print("OK")
'''

code_thresh_2 = '''
def main():
    height = float(input("Height: "))
    if height < 1.5:  # Different threshold!
        print("Too short")
    else:
        print("OK")
'''

is_equiv, reason = programs_equivalent(code_thresh_1, code_thresh_2)
print("\n=== Example 4: Different thresholds (1.6 vs 1.5) ===")
print(f"Equivalent: {is_equiv}")
if not is_equiv:
    print(f"Reason: {reason}")



=== Example 4: Different thresholds (1.6 vs 1.5) ===
Equivalent: False
Reason: Child 1: No matching branch for region (-∞, 1.6)


In [7]:
# Example 5: Equivalent - branches in different order but same regions
code_order_1 = '''
def main():
    height = float(input("Height: "))
    if height >= 1.9:
        print("Above maximum height")
    elif height <= 1.6:
        print("Below minimum height")
    else:
        print("Correct height")
'''

code_order_2 = '''
def main():
    height = float(input("Height: "))
    if height <= 1.6:
        print("Below min height")
    elif height >= 1.9:
        print("Above max height")
    else:
        print("Correct height")
'''

is_equiv, reason = programs_equivalent(code_order_1, code_order_2)
print("\n=== Example 5: Branches in different order (should be equivalent) ===")
print(f"Equivalent: {is_equiv}")
if not is_equiv:
    print(f"Reason: {reason}")



=== Example 5: Branches in different order (should be equivalent) ===
Equivalent: True


## Equivalent Programs with Different Syntax

Let's compare syntactically different but semantically equivalent programs.


In [5]:
# Version A: Uses 'and' operator
code_a = '''
def main():
    h = float(input("Height: "))
    if h > 1.6 and h < 1.9:
        print("OK")
    else:
        print("Not OK")
'''

# Version B: Uses chained comparison
code_b = '''
def main():
    height = float(input("Height: "))
    if 1.6 < height < 1.9:
        print("OK")
    else:
        print("Not OK")
'''

print("=== Program A (uses 'and') ===")
print_cdg(code_a)

print("\n=== Program B (uses chained comparison) ===")
print_cdg(code_b)

print("\n=== Similarity ===")
sim = compare_programs(code_a, code_b)
for k, v in sim.items():
    print(f"  {k}: {v:.2f}")


=== Program A (uses 'and') ===
BodyNode(main)
  InputNode(type=float, prompt="Height: ")
  ControlNode(type=if, branches=2)
    BranchNode(if/elif, region=(1.6, 1.9))
      ActionNode(print, msg="OK")
    BranchNode(else, region=(-∞, 1.6] ∪ [1.9, ∞))
      ActionNode(print, msg="Not OK")

=== Program B (uses chained comparison) ===
BodyNode(main)
  InputNode(type=float, prompt="Height: ")
  ControlNode(type=if, branches=2)
    BranchNode(if/elif, region=(1.6, 1.9))
      ActionNode(print, msg="OK")
    BranchNode(else, region=(-∞, 1.6] ∪ [1.9, ∞))
      ActionNode(print, msg="Not OK")

=== Similarity ===
  structural: 1.00
  behavioral: 1.00
  regional: 1.00
  input: 1.00
  action: 1.00
  overall: 1.00


In [6]:
# Build CDG and inspect structure
builder = CDGBuilder(code_1)
cdg = builder.get_cdg()

print("Root:", cdg)
print("\nChildren:")
for child in cdg.children:
    print(f"  {child}")
    
    if isinstance(child, ControlNode):
        print(f"    Branches:")
        for branch in child.branches:
            print(f"      {branch}")
            for action in branch.children:
                print(f"        -> {action}")


Root: BodyNode(main)

Children:
  InputNode(type=float, prompt="Enter your height in meters: ")
  ControlNode(type=if, branches=3)
    Branches:
      BranchNode(if/elif, region=(-∞, 1.6))
        -> ActionNode(print, msg="Below minimum astronaut height")
      BranchNode(if/elif, region=(1.6, 1.9))
        -> ActionNode(print, msg="Correct height to be an astronaut")
      BranchNode(else, region={1.6} ∪ [1.9, ∞))
        -> ActionNode(print, msg="Above maximum astronaut height")


In [5]:
# Version A
code_a = '''
def main():
    while True:
        height = float(input("Enter your height in meters: "))

        if height > 1.6 and height < 1.9:
            print("Correct height to be an astronaut")
        elif height <= 1.6:
            print("Below minimum astronaut height")
        elif height >= 1.9:
            print("Above maximum astronaut height")
'''


# Version B
code_b = '''
def main():
    your_height = float(input("Enter your height in meters: "))

    if your_height >= 1.9:
        print("Above maximum astronaut height")
    elif your_height <= 1.6:
        print("Below minimum astronaut height")
    else:
        print("Correct height to be an astronaut")
'''


print("=== Program A (uses 'and') ===")
print_cdg(code_a)

print("\n=== Program B (uses chained comparison) ===")
print_cdg(code_b)

print("\n=== Similarity ===")
sim = compare_programs(code_a, code_b)
for k, v in sim.items():
    print(f"  {k}: {v:.2f}")

is_equiv, reason = programs_equivalent(code_a, code_b)
print("\n=== Bool Comparison ===")
print(f"Equivalent: {is_equiv}")
if not is_equiv:
    print(f"Reason: {reason}")

=== Program A (uses 'and') ===
BodyNode(main)
  ControlNode(type=while, branches=0)
    InputNode(type=float, prompt="Enter your height in meters: ", category=height)
    ControlNode(type=if, branches=3)
      BranchNode(if/elif, region=(1.6, 1.9))
        ActionNode(print, msg="Correct height to be an astronaut")
      BranchNode(if/elif, region=(-∞, 1.6])
        ActionNode(print, msg="Below minimum astronaut height")
      BranchNode(if/elif, region=[1.9, ∞))
        ActionNode(print, msg="Above maximum astronaut height")

=== Program B (uses chained comparison) ===
BodyNode(main)
  InputNode(type=float, prompt="Enter your height in meters: ", category=height)
  ControlNode(type=if, branches=3)
    BranchNode(if/elif, region=[1.9, ∞))
      ActionNode(print, msg="Above maximum astronaut height")
    BranchNode(if/elif, region=(-∞, 1.6])
      ActionNode(print, msg="Below minimum astronaut height")
    BranchNode(else, region=(1.6, 1.9))
      ActionNode(print, msg="Correct height to

## Consecutive IF Normalization

The `programs_equivalent` function now normalizes consecutive single-branch `if` statements into a merged structure for comparison. This means:

- `if A: ... if B: ... if C: ...` is treated as equivalent to `if A: ... elif B: ... elif C: ...`

**When the regions are mutually exclusive and the actions match.**


In [2]:
# Example: Three separate IFs vs if-elif-elif (should be equivalent)
code_separate_ifs = '''
def main():
    height = float(input("Enter your height in meters: "))
    
    if (height > 1.6) and (height < 1.9):
        print("Correct height to be an astronaut")
    if (height <= 1.6):
        print("Below minimum astronaut height")
    if (height >= 1.9):
        print("Above maximum astronaut height")
'''

code_if_elif = '''
HEIGHT_MINIMUM = 1.6
HEIGHT_MAXIMUM = 1.9

def main():
    height = float(input("Enter your height in meters: "))

    if height > HEIGHT_MINIMUM and height < HEIGHT_MAXIMUM:
        print("Correct height to be an astronaut")
    elif height >= HEIGHT_MAXIMUM:
        print("Above maximum astronaut height")
    elif height <= HEIGHT_MINIMUM:
        print("Below minimum astronaut height")
'''

print("=== Code with separate IFs ===")
print_cdg(code_separate_ifs)

print("\n=== Code with if-elif-elif ===")
print_cdg(code_if_elif)

print("\n=== Equivalence Check (with consecutive IF normalization) ===")
is_equiv, reason = programs_equivalent(code_separate_ifs, code_if_elif)
print(f"Equivalent: {is_equiv}")
if not is_equiv:
    print(f"Reason: {reason}")
else:
    print("The consecutive IFs are normalized and matched with the if-elif chain!")


=== Code with separate IFs ===
BodyNode(main)
  InputNode(type=float, prompt="Enter your height in meters: ")
  ControlNode(type=if, branches=1)
    BranchNode(if/elif, region=(1.6, 1.9))
      ActionNode(print, msg="Correct height to be an astronaut")
  ControlNode(type=if, branches=1)
    BranchNode(if/elif, region=(-∞, 1.6])
      ActionNode(print, msg="Below minimum astronaut height")
  ControlNode(type=if, branches=1)
    BranchNode(if/elif, region=[1.9, ∞))
      ActionNode(print, msg="Above maximum astronaut height")

=== Code with if-elif-elif ===
BodyNode(main)
  InputNode(type=float, prompt="Enter your height in meters: ")
  ControlNode(type=if, branches=3)
    BranchNode(if/elif, region=(1.6, 1.9))
      ActionNode(print, msg="Correct height to be an astronaut")
    BranchNode(if/elif, region=[1.9, ∞))
      ActionNode(print, msg="Above maximum astronaut height")
    BranchNode(if/elif, region=(-∞, 1.6])
      ActionNode(print, msg="Below minimum astronaut height")

=== Equi